In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle, islice
from matplotlib.ticker import FuncFormatter

In [ ]:
analysis = '../analysis/'
data = '../data/'
data_out = '../results/data_out/'
output_folder = '../results/data_out/results/'
fig_folder = '../results/figs/'
version = '3_oct_2022'
ylim = (-250000, 700000)
yrs = np.arange(2021, 2050, 1)

In [ ]:
ind_agg = pd.read_csv('../results/data_out/Data_out_names/ind_sum_aggr.csv', index_col=0)['label aggr']

In [ ]:
short_ind_names = {'Agriculture, Forestry, Fishing and Hunting': 'Agriculture',
 'Mining, Quarrying, and Oil and Gas Extraction': 'Mining (incl. fossil fuels)',
 'Utilities': 'Utilities',
 'Construction': 'Construction',
 'Manufacturing': 'Manufacturing',
 'Wholesale Trade': 'Wholesale',
 'Retail Trade': 'Retail',
 'Transportation and Warehousing': 'Transportation',
 'Information': 'Information',
 'Finance and Insurance': 'Finance',
 'Real Estate and Rental and Leasing': 'Real Estate',
 'Professional, Scientific, and Technical Services': 'Professional Services',
 'Management of Companies and Enterprises': 'Company Management',
 'Administrative and Support and Waste Management and Remediation Services': 'Administrative Services',
 'Educational Services': 'Education',
 'Health Care and Social Assistance': 'Health Care',
 'Arts, Entertainment, and Recreation': 'Arts',
 'Accommodation and Food Services': 'Accommodation and Food',
 'Other Services (except Public Administration)': 'Other Services',
 'Public Administration': 'Public Administration'}

In [ ]:
np.unique(ind_agg)

In [ ]:
df_full = pd.DataFrame()
for yr in yrs:
    yr_name = str(yr-1) + '-'+ str(yr)
    df = pd.read_csv(output_folder + 'industry_employment_effects_' + \
                                        yr_name + \
                                         '__' + version + '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

print(df_full.sum().sum())
df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)


utils = set(df_try) - set(ind_agg.index)

utils = dict(zip(utils, ['Utilities' for x in utils]))

ind_agg = pd.concat([ind_agg, pd.Series(utils, dtype=str)])
ind_agg = ind_agg.map(short_ind_names)

indcat = np.unique(ind_agg)
my_colors = list(islice(cycle(['grey',  'darkgrey', 'darkgrey', 'grey', 'dimgrey', 
                               'b', 'darkgrey', 'grey', 'dimgrey', 'darkgrey', 'gold',
                               'r', 'grey', 'c', 'grey', 
                               'dimgrey', 'darkgrey', 'm', 
                               'g', 'grey', 'grey']), None, 
                        len(indcat)))
# ['lime',  'violet', 'y', 'darkred', 'aquamarine', 
#                                'b', 'r', 'pink', 'peru', 'k', 'grey',
#                                'slategrey', 'midnightblue', 'salmon', 'goldenrod', 
#                                'teal', 'purple', 
#                                'deeppink', 'g', 'm', 'c'] 
colordict = dict(zip(indcat, my_colors))
colorlist = [colordict[ind_agg.to_dict()[ind]] for ind in df_try.columns]

pd.concat([df_try.cumsum().clip(lower=0), df_try.cumsum().clip(upper=0)], axis=1).\
            plot.area(color=colorlist, figsize=(6,4), ylim=ylim, legend='reverse')#.\
                #legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

colordict_select = {a:b for a,b in zip(colordict.keys(), colordict.values()) if b not in ['grey', 'dimgrey', 'darkgrey']}

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colordict_select.values()]
plt.legend(markers, colordict_select.keys(), numpoints=1, loc='center left', bbox_to_anchor=(0,-0.3), ncols=2)

# plt.vlines(x=[2034, 2038], ymin=[-50000, -250000], ymax=[700000, 700000], colors='teal', ls='--', lw=2)
# plt.text(2025, -120000, 'Transition phase', fontsize=13)
# plt.text(2024, 400000, 'Scale up', fontsize=13)
# plt.text(2034.2, 600000, 'Scale \n down', fontsize=13)
# plt.text(2038.6, 400000, 'Low carbon power \nsystem', fontsize=13)

# despine
sns.despine()

# set ylim
ylim = [-150000, 700000]
plt.ylim(ylim)

# fewer yticks
plt.yticks(np.arange(0, 700000, 200000))

# add thousand separator to y axis
ax = plt.gca()
ax.get_yaxis().set_major_formatter(
    FuncFormatter(lambda x, p: format(int(x), ',')))


plt.ylabel('Cumulative worker demand \nin the electricity supply chain')     

# save figure
plt.savefig(fig_folder + 'cumulative_worker_demand_industry.png', bbox_inches='tight', dpi=300)
# save as pdf
plt.savefig(fig_folder + 'cumulative_worker_demand_industry.pdf', bbox_inches='tight', dpi=300)
# save as svg
plt.savefig(fig_folder + 'cumulative_worker_demand_industry.svg', bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
df_try.cumsum().clip(lower=0).sum(axis=1)#, df_try.cumsum().clip(upper=0)

In [ ]:
major_occ = pd.read_csv('../results/data_out/results_sensitivity/annual_change_sens/4_empl_95% by 2035_2021-2022.csv', index_col=0)['OCC_TITLE_major']

In [ ]:
cols = ['Wind-capex', 'Solar-capex',
       'Natural gas-capex', 'Coal-capex', 'Biomass-capex', 'Geothermal-capex',
       'Hydro-capex', 'Battery storage-capex', 'T&D-capex', 'Biomass-opex', 'Coal-opex',
       'Solar-opex', 'Natural gas-opex', 'Geothermal-opex', 'Hydro-opex',
       'Nuclear-opex', 'Wind-opex', 'Transmission and distribution-opex',
       'Battery storage-opex']

In [ ]:
short_major_occ_names = {'Management Occupations': 'Management',
 'Business and Financial Operations Occupations': 'Business & Financial',
 'Computer and Mathematical Occupations': 'Computer & Maths',
 'Architecture and Engineering Occupations': 'Engineering',
 'Life, Physical, and Social Science Occupations': 'Science Occupations',
 'Community and Social Service Occupations': 'Social Service',
 'Legal Occupations': 'Legal',
 'Education, Training, and Library Occupations': 'Education',
 'Arts, Design, Entertainment, Sports, and Media Occupations': 'Arts',
 'Healthcare Practitioners and Technical Occupations': 'Healthcare Pract.',
 'Healthcare Support Occupations': 'Healthcare Support',
 'Protective Service Occupations': 'Protective Service',
 'Food Preparation and Serving Related Occupations': 'Food Preparation',
 'Building and Grounds Cleaning and Maintenance Occupations': 'Cleaning and Maintenance ',
 'Personal Care and Service Occupations': 'Personal Care',
 'Sales and Related Occupations': 'Sales',
 'Office and Administrative Support Occupations': 'Administrative',
 'Farming, Fishing, and Forestry Occupations': 'Farming',
 'Construction and Extraction Occupations': 'Construction & Extraction',
 'Installation, Maintenance, and Repair Occupations': 'Installation & Repair',
 'Production Occupations': 'Production',
 'Transportation and Material Moving Occupations': 'Transportation'}

In [ ]:
df_full = pd.DataFrame()
for yr in yrs:
    yr_name = str(yr-1) + '-'+ str(yr)
    df = pd.read_csv(output_folder + 'diff_baseline_employment_effects_' + \
                                        yr_name + \
                                         '__' + version + '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.loc[:,cols].unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)

short_majocc = major_occ.map(short_major_occ_names)

occcat = np.unique(short_majocc)
my_colors = list(islice(cycle(['b', 'grey', 'g', 'grey', 'darkgrey', 'm', 'grey', 'lime', 
                               'darkgrey', 'dimgrey', 'grey', 'darkgrey', 
                               'teal', 'grey', 'violet', 'darkgrey', 'peru', 'grey', 
                               'deeppink', 'dimgrey', 'grey', 'aquamarine']), None, 
                        len(occcat)))
colordict = dict(zip(occcat, my_colors))
colorlist = [colordict[occ] for occ in short_majocc]


pd.concat([df_try.cumsum().clip(lower=0), df_try.cumsum().clip(upper=0)], axis=1).plot.area(color=colorlist, 
                figsize=(6,4), ylim=ylim, legend='reverse')#.\
                #legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

colordict_select = {a:b for a,b in zip(colordict.keys(), colordict.values()) if b not in ['grey', 'dimgrey', 'darkgrey']}

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colordict_select.values()]
plt.legend(markers, colordict_select.keys(), numpoints=1, loc='center left', bbox_to_anchor=(0, -0.35), ncols=2)

# plt.vlines(x=[2034, 2038], ymin=[-8000, -250000], ymax=[700000, 700000], colors='teal', ls='--', lw=2)
# plt.text(2025, -120000, 'Transition phase', fontsize=13)
# plt.text(2024, 400000, 'Scale up', fontsize=13)
# plt.text(2034.2, 600000, 'Scale \n down', fontsize=13)
# plt.text(2038.6, 400000, 'Low carbon power \nsystem', fontsize=13)

# despine
sns.despine()

# set ylim
ylim = [-150000, 700000]
plt.ylim(ylim)

# fewer yticks
plt.yticks(np.arange(0, 700000, 200000))

plt.ylabel('Cumulative worker demand in the electricity supply chain')     

# save figure
plt.savefig(fig_folder + 'cumulative_worker_demand_occ.png', bbox_inches='tight', dpi=300)
# save as pdf
plt.savefig(fig_folder + 'cumulative_worker_demand_occ.pdf', bbox_inches='tight', dpi=300)
# save as svg
plt.savefig(fig_folder + 'cumulative_worker_demand_occ.svg', bbox_inches='tight', dpi=300)


plt.show()

In [ ]:
df_full = pd.DataFrame()
for yr in yrs:
    yr_name = str(yr-1) + '-'+ str(yr)
    df = pd.read_csv(output_folder + 'diff_baseline_employment_effects_' + \
                                        yr_name + \
                                         '__' + version + '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.loc[:,cols].unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)

short_majocc = major_occ.map(short_major_occ_names)

occcat = np.unique(short_majocc)
my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k', 'm', 'c', 'lime', 
                               'slategrey', 'midnightblue', 'salmon', 'goldenrod', 
                               'teal', 'purple', 'violet', 'pink', 'peru', 'grey', 
                               'deeppink', 'olive', 'darkred', 'aquamarine']), None, 
                        len(occcat)))
colordict = dict(zip(occcat, my_colors))
colorlist = [colordict[occ] for occ in short_majocc]


pd.concat([df_try.cumsum().clip(lower=0), df_try.cumsum().clip(upper=0)], axis=1).plot.area(color=colorlist, 
                figsize=(6,4), ylim=ylim, legend='reverse')#.\
                #legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colordict.values()]
plt.legend(markers, colordict.keys(), numpoints=1, loc='center left', bbox_to_anchor=(0, -0.7), ncols=2)

plt.vlines(x=[2034, 2038], ymin=[-8000, -250000], ymax=[700000, 700000], colors='teal', ls='--', lw=2)
plt.text(2025, -120000, 'Transition phase', fontsize=13)
plt.text(2024, 400000, 'Scale up', fontsize=13)
plt.text(2034.2, 580000, 'Scale \n down', fontsize=13)
plt.text(2038.6, 400000, 'Low carbon power \nsystem', fontsize=13)

# despine
sns.despine()

# set ylim
ylim = [-150000, 700000]
plt.ylim(ylim)

# fewer yticks
plt.yticks(np.arange(0, 700000, 200000))

plt.ylabel('Cumulative worker demand in the electricity supply chain')     

# save figure
plt.savefig(fig_folder + 'cumulative_worker_demand_occ.png', bbox_inches='tight', dpi=300)
# save as pdf
plt.savefig(fig_folder + 'cumulative_worker_demand_occ.pdf', bbox_inches='tight', dpi=300)
# save as svg
plt.savefig(fig_folder + 'cumulative_worker_demand_occ.svg', bbox_inches='tight', dpi=300)


plt.show()